# Preprocessing

This notebook is just to make clear my procedure for processing the data. Since the California data set is around 33 million data points, it is too large to work with directly for computational reasons. However, there is also the issue that a lot of statistical tests and standard error estimates become meaningless at that level of data. In a prior notebook, I took a stratified subsample of the data which will be used for modelling purposes.

### The data
The data consist of, beyond a stratified subsample of the original traffic stop data, census level data of california and its counties. Our business here is to add county-level predictors to the data and then export that data out as a `.csv` so we have more information with which we can model. The traffic stop data begins in 2009 and ends in 2016. 

All data will be uploaded to Github for verification but the county level data may be obtained here:

http://www.dof.ca.gov/Forecasting/Demographics/Projections/

and

https://data.ca.gov/dataset/california-population-projection-county-age-gender-and-ethnicity

The data is estimated up to the current year, with projections beyond that. It is essentially "as is". The only modification I made was to the demographics data - it was an excel file and I had to simply reshape so that it would read in nicely.

In [1]:
library(dplyr)
library(ggplot2)

Warning message:
"package 'dplyr' was built under R version 3.5.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'ggplot2' was built under R version 3.5.2"

In [2]:
f <- 'stratifiedSample.csv'
train <- read.csv(f)

#We actually don't use this particular file at all, but it's nice to have I suppose.
f <- 'population.csv'
pop_data <- read.csv(f)

f <- 'CAdemographics.csv'
demo_data <-  read.csv(f, stringsAsFactors = F)



The individual level california stop data. It has been stratified so that the sub sample is in proportion to the original. While it is much smaller than the original data (around 4% of the original), we are still in excess of a million data points, so we shouldn't lose too much information. The reason for this is mainly computational - most packages in `R` simply cannot account for that volume of data. Under some assumptions, the estimates from the subsample shouldn't be drastically different than if we had estimated on the entire data set.

In [3]:
head(train)

X,stop_date,county_name,driver_gender,driver_age_raw,driver_race,violation_raw,search_conducted,search_type,contraband_found,stop_outcome,is_arrested,count,year,month,year_c
1,Jul 2009,San Diego County,1,33-39,Hispanic,Moving Violation (VC),0,NA,0,CHP 215,0,1,2009,7,0
2,Jul 2009,Shasta County,1,49+,White,Motorist/Public Service,0,NA,0,Motorist/Public Service,0,1,2009,7,0
3,Jul 2009,Shasta County,1,25-32,White,Moving Violation (VC),0,NA,0,CHP 215,0,1,2009,7,0
4,Jul 2009,Shasta County,1,15-25,White,Moving Violation (VC),0,NA,0,CHP 215,0,1,2009,7,0
5,Jul 2009,Mariposa County,0,40-48,White,Moving Violation (VC),0,NA,0,CHP 215,0,1,2009,7,0
6,Jul 2009,Los Angeles County,0,25-32,Black,Mechanical or Nonmoving Violation (VC),0,NA,0,CHP 215,0,1,2009,7,0


This just consists of some population data for each county in California. We are interested primarily in two things from here: the proportion of the total population each county makes up, and the proportion of traffic stops we have in relation to the county populations. I.e., are there counties over represented in our data?

In [4]:
head(pop_data)

fips,county,year,age,pop_female,pop_male,pop_total
6001,ALAMEDA,1970,0,8533,8671,17204
6001,ALAMEDA,1970,1,8151,8252,16403
6001,ALAMEDA,1970,2,7753,8015,15768
6001,ALAMEDA,1970,3,8018,8412,16430
6001,ALAMEDA,1970,4,8551,8648,17199
6001,ALAMEDA,1970,5,9012,9208,18220


In [5]:
head(demo_data)

County,Race,X2010,X2011,X2012,X2013,X2014,X2015,X2016,X2017,...,X2051,X2052,X2053,X2054,X2055,X2056,X2057,X2058,X2059,X2060
Alameda County,White (Non-Hispanic),518930,520584,525521,529819,535908,540150,542669,544721,...,662269,665732,669384,673244,677058,681318,685690,690083,694571,699114
Alameda County,Black (Non-Hispanic),185086,184967,185536,186106,187037,187229,186639,186372,...,183564,183103,182583,182020,181544,180988,180589,180131,179750,179308
Alameda County,AIAN (Non-Hispanic),4190,4262,4330,4404,4485,4567,4627,4733,...,7210,7253,7327,7379,7451,7502,7577,7624,7688,7797
Alameda County,Asian (Non-Hispanic),396745,404090,412295,420454,428974,437425,444880,450400,...,632260,635438,638817,641838,644653,647369,650079,652805,655391,657809
Alameda County,NHPI (Non-Hispanic),12357,12582,12848,13090,13316,13577,13762,13901,...,18754,18852,18881,18957,19044,19102,19119,19170,19241,19276
Alameda County,MR (Non-Hispanic),57045,58192,59642,61066,62560,63726,64765,65868,...,110303,111513,112571,113784,114904,116028,117264,118353,119587,120499


Let's add some county level predictors to California. 

We will do this simply by aggregating the population and demographics data so that it matches the time and county of our traffic stop data, then add new columns to the traffics stop data. We will add total population of each county, stratified by year, sex, and ethnicity. The raw numbers will be useful later, but for the model building, we will convert these to proportions later. Since we want to stay restricted to the driving ages within out data, we wont include anyone younger than 15 years old. To get the demographic estimates, we will find the proportion of each race in the county, and simply multiply that proportion with our driving age population. This will serve as an estimator for the driving population in that county.

First, just group the population data up by so that we get a female, male and total population for each year, in each county.

In [6]:
county_populations <- demo_data %>%
                        group_by(County) %>%
                            summarise(pop2010 = sum(X2010, na.rm = T),
                                      pop2011 = sum(X2011, na.rm = T),
                                      pop2012 = sum(X2012, na.rm = T),
                                      pop2013 = sum(X2013, na.rm = T),
                                      pop2014 = sum(X2014, na.rm = T),
                                      pop2015 = sum(X2015, na.rm = T),
                                      pop2016 = sum(X2016, na.rm = T))

head(county_populations)

Warning message:
"package 'bindrcpp' was built under R version 3.5.2"

County,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016
Alameda County,1515354,1532186,1554698,1576023,1600366,1621005,1637176
Alpine County,1175,1169,1162,1159,1155,1150,1128
Amador County,38069,37894,37746,37582,37448,37313,37181
Butte County,220193,220563,221708,222083,223727,224180,224761
Calaveras County,45535,45414,45305,45116,45010,44899,44747
Colusa County,21465,21626,21781,21972,22139,22271,22428


In [7]:
county_demographics <- demo_data %>%
                        group_by(County, Race) %>%
                            summarise(pop2010 = sum(X2010, na.rm = T),
                                      pop2011 = sum(X2011, na.rm = T),
                                      pop2012 = sum(X2012, na.rm = T),
                                      pop2013 = sum(X2013, na.rm = T),
                                      pop2014 = sum(X2014, na.rm = T),
                                      pop2015 = sum(X2015, na.rm = T),
                                      pop2016 = sum(X2016, na.rm = T))

head(county_demographics, nrow = 10)

County,Race,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016
Alameda County,AIAN (Non-Hispanic),4190,4262,4330,4404,4485,4567,4627
Alameda County,Asian (Non-Hispanic),396745,404090,412295,420454,428974,437425,444880
Alameda County,Black (Non-Hispanic),185086,184967,185536,186106,187037,187229,186639
Alameda County,Hispanic (any race),341001,347509,354526,361084,368086,374331,379834
Alameda County,MR (Non-Hispanic),57045,58192,59642,61066,62560,63726,64765
Alameda County,NHPI (Non-Hispanic),12357,12582,12848,13090,13316,13577,13762


The following piece of code is not very pretty or optimized, but it gets the job done - we are just going to population the `train` dataframe with county level predictors extracted from the our demographics data frames. 

In [8]:
counties_train <- unique(train$county_name)
counties_demo <- unique(county_demographics$County)
training_years <- c(2010:2016)

train$population <- NA
train$black_pop <- NA
train$white_pop <- NA
train$hispanic_pop <- NA
train$asian_pop <- NA

black_id <- 'Black (Non-Hispanic)'
white_id <- 'White (Non-Hispanic)'
asian_id <- 'Asian (Non-Hispanic)'
hispanic_id <- 'Hispanic (any race)'

for (county in counties_train) {
    
    if (county %in% counties_demo) {
        
        for (year in training_years) {
            
            demo_idx <- paste0('pop', year)
            
            train[train$county_name == county
                  & train$year == year, ]$population <- as.numeric(county_populations[county_populations$County == county, demo_idx])
            
            train[train$county_name == county
                  & train$year == year, ]$black_pop <- as.numeric(county_demographics[county_demographics$County == county &
                                                                                      county_demographics$Race == black_id,
                                                                                      demo_idx])
            train[train$county_name == county
                  & train$year == year, ]$white_pop <- as.numeric(county_demographics[county_demographics$County == county &
                                                                                      county_demographics$Race == white_id,
                                                                                      demo_idx])
            train[train$county_name == county
                  & train$year == year, ]$asian_pop <- as.numeric(county_demographics[county_demographics$County == county &
                                                                                      county_demographics$Race == asian_id,
                                                                                      demo_idx])
            train[train$county_name == county
                  & train$year == year, ]$hispanic_pop <- as.numeric(county_demographics[county_demographics$County == county &
                                                                                      county_demographics$Race == hispanic_id,
                                                                                      demo_idx])
        }
    }
}

We don't have 2009 data, but upon a quick google search, we can find that California's population change was about 1% from 2009 to 2010, so assuming that most counties saw equal growth, we can just take 99% of 2010's population to fill in the 2009 populations.

In [9]:
for (county in counties_train) {
    train[train$year == 2009 &
          train$county_name == county,]$population <- round(0.99 * mean(train[train$year == 2010 & 
                                                                        train$county_name == county,]$population))
    
    train[train$year == 2009 &
          train$county_name == county,]$white_pop <- round(0.99 * mean(train[train$year == 2010 & 
                                                                             train$county_name == county,]$white_pop))
    train[train$year == 2009 &
          train$county_name == county,]$black_pop <- round(0.99 * mean(train[train$year == 2010 & 
                                                                             train$county_name == county,]$black_pop))
    train[train$year == 2009 &
          train$county_name == county,]$asian_pop <- round(0.99 * mean(train[train$year == 2010 & 
                                                                             train$county_name == county,]$asian_pop))
    train[train$year == 2009 &
          train$county_name == county,]$hispanic_pop <- round(0.99 * mean(train[train$year == 2010 & 
                                                                             train$county_name == county,]$hispanic_pop))

    
}
head(train)

X,stop_date,county_name,driver_gender,driver_age_raw,driver_race,violation_raw,search_conducted,search_type,contraband_found,...,is_arrested,count,year,month,year_c,population,black_pop,white_pop,hispanic_pop,asian_pop
1,Jul 2009,San Diego County,1,33-39,Hispanic,Moving Violation (VC),0,NA,0,...,0,1,2009,7,0,3070026,146377,1492582,984694,328474
2,Jul 2009,Shasta County,1,49+,White,Motorist/Public Service,0,NA,0,...,0,1,2009,7,0,175354,1362,145009,14774,4255
3,Jul 2009,Shasta County,1,25-32,White,Moving Violation (VC),0,NA,0,...,0,1,2009,7,0,175354,1362,145009,14774,4255
4,Jul 2009,Shasta County,1,15-25,White,Moving Violation (VC),0,NA,0,...,0,1,2009,7,0,175354,1362,145009,14774,4255
5,Jul 2009,Mariposa County,0,40-48,White,Moving Violation (VC),0,NA,0,...,0,1,2009,7,0,18063,94,15330,1618,153
6,Jul 2009,Los Angeles County,0,25-32,Black,Mechanical or Nonmoving Violation (VC),0,NA,0,...,0,1,2009,7,0,9740383,817333,2724568,4647871,1328676


Check and make sure that the missing population data matches the missing county data

In [10]:
sum(is.na(train$population))

[1] 93399

In [11]:
dim(train[train$county_name == 'Missing',])[1]

[1] 93399

So all that remains is to impute the Missing County variables. We will treat the field `Missing` as its own county, since we have no reasonable way to impute the county level predictors from the individual level data (The number of stops and racial breakdown within the missing county may not be representative of the overall demographics - we have yet to explore this).

We really just have two options - either use the median population estimates as a robust estimate of the missing county, or look at the population difference between california and the sum of the populations from the counties we *do* have. By the way, what proportion of our data has a missing county?

In [12]:
sum(is.na(train$population)) / dim(train)[1]

[1] 0.07360941

About 7%. If the data is missing completely at random (MCAR) or even missing at random (MAR) then this is fairly inconsequential, as the missing county doesn't carry a lot of information relative to the other data. If there is a non-random mechanism for missingness, then we have more of an issue. We *could* just drop the data, but that is always unsatisfying. We might want to get an idea if the Missing County is relatively homogenous with the rest of the data or if it has some serious systemic differences. Since we mostly don't want to bias our race results, we can take a look the racial breakdown 

(Also, the aggregated data will be useful later)

We'll group by county and race, and calculate the number of traffic stops, number of searches, and the number of contraband "hits" found in the data. We have missing contraband data that can't be reasonably be imputed (it's an outcome in our view, not a predictor, so imputed hit rates would lead to a large amount of bias) and it's also not missing at random:

In [13]:
unique(train[is.na(train$contraband_found),]$search_type)

[1] Incident to Arrest Inventory          Protective Frisk   Warrant           
[5] Other             
7 Levels: Consent Incident to Arrest Inventory Other ... Warrant

In [14]:
unique(train[!is.na(train$contraband_found),]$search_type)

[1] <NA>           Probable Cause Consent       
7 Levels: Consent Incident to Arrest Inventory Other ... Warrant

Only the `Probable Cause` and `Consent` search types have contraband data that's not missing, and note that there is no intersection in the search types between the missing and non-missing contraband data. So clearly, this is not a random mechanism, which means we can't really impute it even if we wanted to. We'll explore this deeper in a different notebook. For now, we'll just take solace in the fact that our `Probable Cause` and `Consent` search types have no missing data besides possibly the `county_name`, and we can aggregate our data so we can get a big picture idea of what is going on at each county. 

By the way `CHP 215` is "Notice to Appear" in court. I.e., it's a ticket. `CHP 281` is a "fix it or ticket". I.e, fix your tail light.

In [16]:
traffic_count_data <- train %>%
                        group_by(year, county_name, driver_race) %>%
                            summarise(total_county_pop = median(population),
                                      county_black = median(black_pop),
                                      county_white = median(white_pop),
                                      county_hispanic = median(hispanic_pop),
                                      county_asian = median(asian_pop),
                                      stops = n(),
                                      num_males = sum(driver_gender),
                                      suspect_dui = sum(violation_raw == 'DUI Check'),
                                      move_violation = sum(violation_raw == 'Moving Violation (VC)'),
                                      searches = sum(search_conducted),
                                      consent = sum((search_type == 'Consent'), na.rm = T),
                                      probable = sum((search_type == 'Probable Cause'), na.rm = T),
                                      hits = sum(contraband_found, na.rm = T),
                                      arrests = sum(is_arrested),
                                      warnings = sum((stop_outcome == 'Verbal Warning')),
                                      chp215 = sum((stop_outcome == 'CHP 215')),
                                      collisions = sum((stop_outcome == 'Traffic Collision')),
                                      chp281 = sum((stop_outcome == 'CHP 281')))
head(traffic_count_data)

year,county_name,driver_race,total_county_pop,county_black,county_white,county_hispanic,county_asian,stops,num_males,...,move_violation,searches,consent,probable,hits,arrests,warnings,chp215,collisions,chp281
2009,Alameda County,Asian,1500200,183235,513741,337591,392778,560,381,...,215,17,0,0,0,12,101,258,45,11
2009,Alameda County,Black,1500200,183235,513741,337591,392778,813,480,...,250,61,0,1,1,40,127,421,36,10
2009,Alameda County,Hispanic,1500200,183235,513741,337591,392778,1012,795,...,283,62,0,4,3,41,121,454,60,31
2009,Alameda County,Other,1500200,183235,513741,337591,392778,504,382,...,204,13,0,0,0,9,46,298,31,12
2009,Alameda County,White,1500200,183235,513741,337591,392778,1588,1096,...,564,27,0,2,2,17,229,750,107,33
2009,Alpine County,Asian,1163,0,891,64,8,14,12,...,10,0,0,0,0,0,4,8,0,1


Note that I've just added the total populations to the above data frame just so that they're in one place.

For the most part, this particular notebook is done. We will export our work out as two `.csv` files so that in a future notebook, we can begin modeling the data. 

In [17]:
write.csv(traffic_count_data, 'trafficCountData.csv', row.names = F)
write.csv(train, 'trafficWCountyPredictors.csv', row.names = F)